In [182]:
import pandas as pd
import numpy as np
import re

In [183]:
# # Load the dataset
# df = pd.read_csv("dirty_financial_transactions.csv")

# # Take 1000 random rows
# sample_df = df.sample(n=1000, random_state=42).reset_index(drop=True)

# # Save to a new CSV file
# sample_df.to_csv("dirty_financial_transactions_1k.csv", index=False)

# print("✅ Saved 1000-row sample to dirty_financial_transactions_1k.csv")

✅ Saved 1000-row sample to dirty_financial_transactions_1k.csv


In [178]:
# import pandas as pd
# import numpy as np
# import re

# # ===============================================================
# # 1️⃣ HANDLE MISSING VALUES
# # ===============================================================
# def handle_missing_values(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.copy()
#     # Drop rows where ID fields are null
#     df = df.dropna(subset=["Transaction_ID", "Customer_ID"])
    
#     for col in df.columns:
#         if col in ["Transaction_ID", "Customer_ID"]:
#             continue
#         if df[col].dtype in [np.float64, np.int64]:
#             df[col] = df[col].fillna(df[col].median())
#         else:
#             df[col] = df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else "Unknown")
#     return df

# # # =============================================================== Does not have any duplicates
# # # 2️⃣ REMOVE DUPLICATES
# # # ===============================================================
# # def remove_duplicates(df: pd.DataFrame) -> pd.DataFrame:
# #     return df.drop_duplicates().copy()

# # ===============================================================
# # 3️⃣ HANDLE INVALID VALUES (TYPE & RANGE)
# # ===============================================================
# def handle_invalid_values(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.copy()

#     # Remove symbols from Price
#     if "Price" in df.columns:
#         df["Price"] = df["Price"].astype(str).str.replace(r"[^0-9.\-]", "", regex=True)

#     df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
#     df["Price"] = pd.to_numeric(df["Price"], errors="coerce")

#     # Replace negative or zero values with NaN
#     df.loc[df["Quantity"] <= 0, "Quantity"] = np.nan
#     df.loc[df["Price"] <= 0, "Price"] = np.nan

#     # Remove non-integer quantities
#     df.loc[~df["Quantity"].isna(), "Quantity"] = df["Quantity"].apply(
#         lambda x: x if float(x).is_integer() else np.nan
#     )

#     df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
#     df["Price"] = pd.to_numeric(df["Price"], errors="coerce")

#     # Round price to 2 decimals
#     df["Price"] = df["Price"].round(2)

#     return df

# # # =============================================================== No outliers
# # # 4️⃣ HANDLE OUTLIERS
# # # ===============================================================
# # def handle_outliers(df: pd.DataFrame) -> pd.DataFrame:
# #     df = df.copy()
# #     for col in ["Quantity", "Price"]:
# #         if col in df.columns:
# #             Q1 = df[col].quantile(0.25)
# #             Q3 = df[col].quantile(0.75)
# #             IQR = Q3 - Q1
# #             lower = Q1 - 1.5 * IQR
# #             upper = Q3 + 1.5 * IQR
# #             df.loc[~df[col].between(lower, upper), col] = np.nan
# #     return df

# # ===============================================================
# # 5️⃣ STANDARDIZE TEXT CASES
# # ===============================================================
# def standardize_text(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.copy()
#     for col in df.select_dtypes(include="object"):
#         df.loc[:, col] = df[col].astype(str).str.strip().str.lower()
#     return df

# # ===============================================================
# # 6️⃣ NORMALIZE DATE FORMATS
# # ===============================================================
# def normalize_dates(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Normalize the 'Transaction_Date' column:
#     - Convert to pandas datetime, invalid dates become NaT
#     - Keep missing/invalid dates as NaT (do not fill)
#     - Format valid dates as YYYY-MM-DD
#     """
#     df = df.copy()
#     df["Transaction_Date"] = pd.to_datetime(df["Transaction_Date"], errors="coerce")
#     # Format only non-null dates
#     df.loc[~df["Transaction_Date"].isna(), "Transaction_Date"] = df.loc[~df["Transaction_Date"].isna(), "Transaction_Date"].dt.strftime("%Y-%m-%d")
#     return df


# # ===============================================================
# # 7️⃣ STANDARDIZE CATEGORICAL VALUES
# # ===============================================================
# def standardize_categories(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.copy()
#     # Payment methods
#     payment_map = {
#         "paypal": "PayPal",
#         "pay pal": "PayPal",
#         "creditcard": "Credit Card",
#         "credit card": "Credit Card",
#         "cash": "Cash",
#     }
#     # Transaction status
#     status_map = {
#         "completed": "Completed",
#         "complete": "Completed",
#         "pending": "Pending",
#         "failed": "Failed",
#     }
#     # Product names
#     product_map = {
#         "laptop": "Laptop", "lap": "Laptop", "lapto": "Laptop", "lapt": "Laptop", "la": "Laptop", "l": "Laptop",
#         "coffee machine": "Coffee Machine", "coffee mach": "Coffee Machine", "coffee mac": "Coffee Machine",
#         "coffee machi": "Coffee Machine", "coffee": "Coffee Machine", "cof": "Coffee Machine", "c": "Coffee Machine",
#         "tablet": "Tablet", "tabl": "Tablet", "ta": "Tablet", "tab": "Tablet", "table": "Tablet",
#         "smartphone": "Smartphone", "smart": "Smartphone", "smartp": "Smartphone", "smartpho": "Smartphone",
#         "smar": "Smartphone", "sma": "Smartphone", "sm": "Smartphone", "t": "Smartphone", "s": "Smartphone",
#         "headphones": "Headphones", "headph": "Headphones", "headphone": "Headphones", "h": "Headphones"
#     }

#     if "Payment_Method" in df.columns:
#         df.loc[:, "Payment_Method"] = df["Payment_Method"].map(lambda x: payment_map.get(x, x))
#     if "Transaction_Status" in df.columns:
#         df.loc[:, "Transaction_Status"] = df["Transaction_Status"].map(lambda x: status_map.get(x, x))
#     if "Product_Name" in df.columns:
#         df.loc[:, "Product_Name"] = df["Product_Name"].map(lambda x: product_map.get(x, x))
#     return df

# # # =============================================================== No changes to the df
# # # 8️⃣ CLEAN TRANSACTION_ID
# # # ===============================================================
# # def handle_transaction_id(df: pd.DataFrame) -> pd.DataFrame:
# #     df = df.copy()
# #     df["Transaction_ID"] = df["Transaction_ID"].astype(str).str.strip()
# #     # Keep IDs that match Txxxx, otherwise leave as-is
# #     return df

# # # ===============================================================
# # # 9️⃣ FILL MISSING CATEGORICALS
# # # ===============================================================
# # def handle_missing_categories(df: pd.DataFrame) -> pd.DataFrame:
# #     df = df.copy()
# #     for col in ["Product_Name", "Payment_Method", "Transaction_Status"]:
# #         if col in df.columns:
# #             df[col] = df[col].fillna("Unknown")
# #     return df

# # ===============================================================
# # 🧹 MAIN CLEANING PIPELINE
# # ===============================================================
# def clean_data(df: pd.DataFrame) -> pd.DataFrame:
#     df = handle_missing_values(df)
#     # df = remove_duplicates(df)
#     df = handle_invalid_values(df)
#     # df = handle_outliers(df)
#     df = standardize_text(df)
#     df = normalize_dates(df)
#     df = standardize_categories(df)
#     df = handle_transaction_id(df)
#     df = handle_missing_categories(df)
#     return df

# # ===============================================================
# # ✅ EXECUTE PIPELINE
# # ===============================================================
# df = pd.read_csv("dirty_financial_transactions_500.csv")
# cleaned_df = clean_data(df)

# # Save cleaned dataset
# cleaned_df.to_csv("cleaned_financial_transactions.csv", index=False)

# print("✅ Data cleaning complete! Saved to 'cleaned_financial_transactions.csv'")
# print(cleaned_df.head())
# print(f"Total rows after cleaning: {len(cleaned_df)}")


In [184]:

# ===============================================================
# 1️⃣ HANDLE MISSING VALUES
# ===============================================================
def handle_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Handle missing values:
    - Drop rows with missing Transaction_ID or Customer_ID.
    - Clean and convert 'Price' and 'Quantity' to numeric before filling.
    - Fill numeric columns with median.
    - Fill categorical/text columns with mode.
    Prints rows where changes occur for clarity.
    """
    df = df.copy()

    # 🔹 Identify rows with missing IDs (to drop)
    missing_ids = df[df["Transaction_ID"].isna() | df["Customer_ID"].isna()]
    if not missing_ids.empty:
        print("Rows with missing Transaction_ID or Customer_ID (will be dropped):")
        print(missing_ids)
    
    # Drop invalid ID rows
    df = df.dropna(subset=["Transaction_ID", "Customer_ID"])

    # 🔹 Special cleanup for numeric-like object columns (e.g., 'Price', 'Quantity')
    numeric_like_cols = ["Price", "Quantity"]
    for col in numeric_like_cols:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype(str)
                .str.replace(r"[^0-9.\-]", "", regex=True)
            )
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # 🔹 Fill missing values
    for col in df.columns:
        if col in ["Transaction_ID", "Customer_ID", "Transaction_Date"]:
            continue  # IDs handled separately and skip Date

        if pd.api.types.is_numeric_dtype(df[col]):
            null_rows = df[df[col].isna()]
            if not null_rows.empty:
                print(f"\nRows where numeric column '{col}' was filled with median:")
                print(null_rows)
            df[col] = df[col].fillna(df[col].median())

        else:
            null_rows = df[df[col].isna()]
            if not null_rows.empty:
                print(f"\nRows where categorical/text column '{col}' was filled with mode:")
                print(null_rows)
            df[col] = df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else "Unknown")

    return df



# ===============================================================
# 2️⃣ HANDLE INVALID VALUES (TYPE & RANGE)
# ===============================================================
def handle_invalid_values(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Remove symbols from Price
    if "Price" in df.columns:
        df["Price"] = df["Price"].astype(str).str.replace(r"[^0-9.\-]", "", regex=True)

    df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
    df["Price"] = pd.to_numeric(df["Price"], errors="coerce")

    # Show rows before changes
    neg_quantity_rows = df[df["Quantity"] <= 0]
    neg_price_rows = df[df["Price"] <= 0]
    non_int_quantity_rows = df[~df["Quantity"].dropna().apply(float.is_integer)]

    if not neg_quantity_rows.empty:
        print("\nRows with negative or zero Quantity (will be set to NaN):")
        print(neg_quantity_rows)
    if not neg_price_rows.empty:
        print("\nRows with negative or zero Price (will be set to NaN):")
        print(neg_price_rows)
    if not non_int_quantity_rows.empty:
        print("\nRows with non-integer Quantity (will be set to NaN):")
        print(non_int_quantity_rows)

    # Replace negative or zero values with NaN
    df.loc[df["Quantity"] <= 0, "Quantity"] = np.nan
    df.loc[df["Price"] <= 0, "Price"] = np.nan

    # Remove non-integer quantities
    df.loc[~df["Quantity"].isna(), "Quantity"] = df["Quantity"].apply(
        lambda x: x if float(x).is_integer() else np.nan
    )

    # Round price to 2 decimals
    df["Price"] = df["Price"].round(2)
    return df

# ===============================================================
# 3️⃣ STANDARDIZE TEXT CASES
# ===============================================================
def standardize_text(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in df.select_dtypes(include="object"):
        df.loc[:, col] = df[col].astype(str).str.strip().str.lower()
    # Show sample of text columns after standardization
    print("\nSample of text columns after standardization:")
    for col in ["Product_Name", "Payment_Method", "Transaction_Status"]:
        if col in df.columns:
            print(df[col].head(5))
    return df

# ===============================================================
# 4️⃣ NORMALIZE DATE FORMATS
# ===============================================================
def normalize_dates(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["Transaction_Date"] = pd.to_datetime(df["Transaction_Date"], errors="coerce")
    # Format only non-null dates
    df.loc[~df["Transaction_Date"].isna(), "Transaction_Date"] = df.loc[~df["Transaction_Date"].isna(), "Transaction_Date"].dt.strftime("%Y-%m-%d")
    # Show sample after normalization
    print("\nSample Transaction_Date column after normalization:")
    print(df["Transaction_Date"].head(5))
    return df

# ===============================================================
# 5️⃣ STANDARDIZE CATEGORICAL VALUES
# ===============================================================
def standardize_categories(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    payment_map = {
        "paypal": "PayPal", "pay pal": "PayPal",
        "creditcard": "Credit Card", "credit card": "Credit Card",
        "cash": "Cash",
    }
    status_map = {
        "completed": "Completed", "complete": "Completed",
        "pending": "Pending", "failed": "Failed",
    }
    product_map = {
        "laptop": "Laptop", "lap": "Laptop", "lapto": "Laptop", "lapt": "Laptop", "la": "Laptop", "l": "Laptop",
        "coffee machine": "Coffee Machine", "coffee mach": "Coffee Machine", "coffee mac": "Coffee Machine",
        "coffee machi": "Coffee Machine", "coffee": "Coffee Machine", "cof": "Coffee Machine", "c": "Coffee Machine",
        "tablet": "Tablet", "tabl": "Tablet", "ta": "Tablet", "tab": "Tablet", "table": "Tablet",
        "smartphone": "Smartphone", "smart": "Smartphone", "smartp": "Smartphone", "smartpho": "Smartphone",
        "smar": "Smartphone", "sma": "Smartphone", "sm": "Smartphone", "t": "Smartphone", "s": "Smartphone",
        "headphones": "Headphones", "headph": "Headphones", "headphone": "Headphones", "h": "Headphones"
    }

    if "Payment_Method" in df.columns:
        df.loc[:, "Payment_Method"] = df["Payment_Method"].map(lambda x: payment_map.get(x, x))
        print("\nSample Payment_Method after standardization:")
        print(df["Payment_Method"].head(5))
    if "Transaction_Status" in df.columns:
        df.loc[:, "Transaction_Status"] = df["Transaction_Status"].map(lambda x: status_map.get(x, x))
        print("\nSample Transaction_Status after standardization:")
        print(df["Transaction_Status"].head(5))
    if "Product_Name" in df.columns:
        df.loc[:, "Product_Name"] = df["Product_Name"].map(lambda x: product_map.get(x, x))
        print("\nSample Product_Name after standardization:")
        print(df["Product_Name"].head(5))
    return df

# ===============================================================
# 🧹 MAIN CLEANING PIPELINE
# ===============================================================
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = handle_missing_values(df)
    df = handle_invalid_values(df)
    df = standardize_text(df)
    df = normalize_dates(df)
    df = standardize_categories(df)
    return df

# ===============================================================
# ✅ EXECUTE PIPELINE
# ===============================================================
df = pd.read_csv("dirty_financial_transactions_1k.csv")
cleaned_df = clean_data(df)

# Save cleaned dataset
cleaned_df.to_csv("diy_dataset.csv", index=False)

print("\n✅ Data cleaning complete! Saved to 'diy_dataset.csv'")
print(f"Total rows after cleaning: {len(cleaned_df)}")


Rows with missing Transaction_ID or Customer_ID (will be dropped):
    Transaction_ID Transaction_Date Customer_ID    Product_Name  Quantity  \
36             NaN       2025-02-30       C4919      Smartphone     647.0   
41          T89578       2023-13-01         NaN      Headphones     750.0   
73          T14532       2025-02-30         NaN      Smartphone       7.0   
78          T99090       2020-01-28         NaN  Coffee Machine     915.0   
90             NaN       2021-07-17        C312               T      -2.0   
..             ...              ...         ...             ...       ...   
936            NaN       2023-12-11       C3855      Headphones       NaN   
938            NaN       2025-02-30       C1424          Tablet       8.0   
955         T50615       2023-13-01         NaN          Tablet     -10.0   
985            NaN       2023-13-01       C1969          Tablet     -10.0   
999            NaN       2023-13-01       C1503  Coffee Machine     732.0   

        

In [186]:
# print(df.info())
# print(cleaned_df.info())